How to download data:
- Go to: (https://gemelnet.cma.gov.il/views/dafmakdim.aspx)[https://gemelnet.cma.gov.il/views/dafmakdim.aspx]
- choose the desired Kranot
- Switch to a custom period (instead of 1-year)
- Run the following JS in the console

```javascript
document.getElementById("inout").value=".."
function work(year, month) {
        actualYear = 2018-year
        actualMonth = 1 + month
        console.log("fetching " + actualYear + ", " + actualMonth)
        $("#ddlbHodashimMe")[0].selectedIndex = month
        $("#ddlbHodashimAd")[0].selectedIndex = month
        $("#ddlbShanimMe")[0].selectedIndex = year
        $("#ddlbShanimAd")[0].selectedIndex = year

        if (actualYear == 2018 && actualMonth >= 6) {
            console.log("skipping ...")
            month += 1
            if (month == 12) {
               month = 0
               year += 1
            }
            work(year, month)
        }
        else {
            document.getElementById("cbDisplay").click()
            console.log("submitted")
        }

        setTimeout(function() {
           $find('ReportViewer1').exportReport('CSV');
           console.log("downloaded " + actualYear + ", " + actualMonth)

            month += 1
            if (month == 12) {
               month = 0
               year += 1
            }
            work(year, month)
        }, 70000)
}
work(0, 0)
// earliest supported year is 1999

```


To process all the files and merge to a single table, run the following:

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline  

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
from io import StringIO

def process(filepath):
    with open(filepath, "r") as f:
        lines = f.readlines()

    goodlines = []
    good = False
    for l in lines:
        l = l.strip()
        if not good and l == '':
            good = True
            continue
        if good and l == '':
            break
        if len(l.split(",")) == 1:
            continue
        if good:
            l = l.replace("\n", " ")
            goodlines.append(l)


    TESTDATA = StringIO("\n".join(goodlines))
    df = pd.read_csv(TESTDATA)
    
    df = df.rename({"Textbox89": "month_return", "SHM2": "name", "Textbox90": "active", "ID2": "ID",
               "Textbox77": "liquidity", "Textbox78": "net_flow", "Textbox83": "AUM", "Textbox84": "fee_assets",
               "Textbox103": "fee_deposits", "Textbox85": "sharpe_5y", "Textbox86": "sector_median_sharpe_5y",
               "Textbox87": "yearly_alpha_5y", "Textbox88": "YTD_return"}, axis=1)
    
    df = df[list(c for c in df.columns if "Textbox" not in c)]
    
    dateStr = lines[1].split(",")[0].strip().split(' ')[-1]
    date = pd.to_datetime(dateStr, format="%m/%Y")
    df.loc[:, "month"] = date
    
    return df

In [3]:
import glob

path = "data/part*/*.csv"
alldfs = []
i = 0
for filename in glob.iglob(path, recursive=True):
    df = process(filename)
    alldfs.append(df)
    i += 1
    #if i == 3:
    #    break
    
    
mdf = pd.concat(alldfs)
mdf = mdf.sort_values(["month", "name"])
mdf = mdf.reset_index().drop("index",axis=1)

def markType(filename, typeName):
    df = process(filename)
    kranot = set(df["name"].values)
    mdf.loc[mdf["name"].isin(kranot), "type"] = typeName

markType("data/types/KranotHishtalmut.csv", "קרן השתלמות")
markType("data/types/GemelHashkaa.csv", "קופת גמל להשקעה")
markType("data/types/HisahonYeled.csv", "קופג להשקעה - חסכון לכל ילד")
markType("data/types/MerkazitPitsuim.csv", "מרכזית לפיצויים")
markType("data/types/TagmulimIshitPitsuim.csv", "תגמולים ואישית לפיצויים")
mdf.to_csv("data/GemelNet.csv")
